In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
!pip install --upgrade xlrd #colab bug verson xlrd
!pip install geopandas
!pip install pandas fiona shapely pyproj rtree
!pip install contextily
!pip install pygeos
!pip install topojson
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
#!pip install requests py7zr geopandas openpyxl tqdm s3fs PyYAML xlrd
#!pip install git+https://github.com/inseefrlab/cartiflette@80b8a5a28371feb6df31d55bcc2617948a5f9b1a

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 285.5 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 12.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 2.9 MB/s eta 0:00:00


In [2]:
#test_1 = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/5f71ba43-afc8-43a0-b306-dafe29940f9c", sep = ";", encoding = 'utf-8', engine = 'python')
#le lien du dossier data.gouv : 
# https://www.data.gouv.fr/fr/datasets/effectif-de-patients-par-pathologie-sexe-classe-dage-et-territoire-departement-region/
# methode "classique" pour obtenir un dataset a partir d un lien 
# probleme d encodage d ou les parametres sep, encoding, engine
# A FAIRE : COPIER LA BASE DE FACON LOCALE ET S Y REFERER PAR LA SUITE PAR EFFECTIFS.CSV 

In [59]:
test_2 = pd.read_csv("effectifs.csv", sep = ";", encoding = 'utf-8', engine = 'python')
#commande a utiliser une fois qu on a la base 
#copiee en local 
#la base au prealable telechargee sur ordi perso

In [60]:
test_2.head()
test_2.shape

(4057201, 16)

In [6]:
print("nombre de categories differentes pour niv2 est :"+str(len(test_2['patho_niv2'].unique())))
print("nombre de categories differentes pour niv3 est :"+str(len(test_2['patho_niv3'].unique())))

nombre de categories differentes pour niv2 est :47
nombre de categories differentes pour niv3 est :60


In [7]:
#potentiellement creer 123 colonnes pour pouvoir faire du clustering 
#comparer les clusters obtenus par les caracteristiques d un inidivu pour voir si ca matche ou pas 


In [63]:
data_gen = test_2

In [64]:
print(test_2.shape, data_gen.shape)

(4057201, 16) (4057201, 16)


In [10]:
liste_diff_patho1 = data_gen['patho_niv1'].unique()
liste_diff_patho2 = data_gen['patho_niv2'].unique()
liste_diff_patho3 = data_gen['patho_niv3'].unique()

print(liste_diff_patho1, "\n", liste_diff_patho2, "\n", liste_diff_patho3)

['Traitements du risque vasculaire (hors pathologies)'
 'Traitements psychotropes (hors pathologies)'
 'Séjours en hospitalisation complète pour prise en charge de la Covid-19'
 'Traitement antalgique ou anti-inflammatoire (hors pathologies, traitements, maternité ou hospitalisations)'
 'Maladies respiratoires chroniques (hors mucoviscidose)'
 'Maternité (avec ou sans pathologies)'
 'Pas de pathologies repérées, traitements, maternité, hospitalisations ni traitement antalgique ou anti-inflammatoire'
 'Maladies psychiatriques'
 'Autres affections de longue durée (dont 31 et 32)' 'Cancers'
 'Hospitalisations ponctuelles (avec ou sans pathologies, traitements ou maternité)'
 'Insuffisance rénale chronique terminale' nan 'Diabète'
 'Maladies cardio-neurovasculaires'
 'Maladies du foie ou du pancréas (hors mucoviscidose)'
 'Maladies inflammatoires ou rares ou VIH ou SIDA'
 'Maladies neurologiques ou dégénératives'] 
 ['Traitements hypolipémiants (hors pathologies)' nan
 "Traitements antidép

In [71]:
print(data_gen.shape, data_gen['patho_niv1'].isna().sum())

(4057201, 16) 52921


In [72]:
#Creation d une colonne individus
ind = []
for i in range (1,4057202) : 
    ind.append(i)

In [73]:
ind = pd.DataFrame(ind)
ind = ind.set_axis(['ind'], axis='columns')

In [75]:
data_gen_1 = pd.concat([ind, data_gen], axis = 1)

In [15]:
data_gen_1.head()

,ind,annee,patho_niv1,patho_niv2,patho_niv3,top,cla_age_5,sexe,region,dept,ntop,npop,prev,niveau_prioritaire,libelle_classe_age,libelle_sexe,tri
0,1,2020.0,Traitements du risque vasculaire (hors patholo...,Traitements hypolipémiants (hors pathologies),Traitements hypolipémiants (hors pathologies),top_FHypoLi_med,80-84,1.0,1.0,971,290.0,4210.0,6.912,"2,3",de 80 à 84 ans,hommes,51.0
1,2,2020.0,Traitements du risque vasculaire (hors patholo...,Traitements hypolipémiants (hors pathologies),Traitements hypolipémiants (hors pathologies),top_FHypoLi_med,80-84,1.0,3.0,973,40.0,650.0,5.864,"2,3",de 80 à 84 ans,hommes,51.0
2,3,2020.0,Traitements du risque vasculaire (hors patholo...,Traitements hypolipémiants (hors pathologies),Traitements hypolipémiants (hors pathologies),top_FHypoLi_med,80-84,1.0,4.0,999,410.0,5630.0,7.340,"2,3",de 80 à 84 ans,hommes,51.0
3,4,2020.0,Traitements du risque vasculaire (hors patholo...,Traitements hypolipémiants (hors pathologies),Traitements hypolipémiants (hors pathologies),top_FHypoLi_med,80-84,1.0,6.0,999,20.0,350.0,4.913,"2,3",de 80 à 84 ans,hommes,51.0
4,5,2020.0,Traitements du risque vasculaire (hors patholo...,Traitements hypolipémiants (hors pathologies),Traitements hypolipémiants (hors pathologies),top_FHypoLi_med,80-84,1.0,11.0,75,2960.0,20820.0,14.221,"2,3",de 80 à 84 ans,hommes,51.0


In [87]:
data_gen = data_gen_1

In [90]:
#tentative de compréhension de ce que la variable npop représente
#b = data_gen['npop'].unique()
#b = pd.DataFrame(b)

In [91]:
#b = b.dropna()

In [92]:
#b = b.set_axis(['test'], axis = 'columns')

In [93]:
#b.head()

In [94]:
#sum(b['test'])

In [21]:
#resultat peu concluant

In [22]:
#creation d indicatrices pour futur clustering

In [95]:
data_gen.shape

(4057201, 17)

In [96]:
type(liste_diff_patho3)

list

In [97]:
liste_diff_patho3 = list(liste_diff_patho3)

In [98]:
type(liste_diff_patho3)

list

In [99]:
temporaire = []
for i in data_gen['patho_niv3'] : 
    if i == 'Syndrome coronaire aigu' :
        temporaire.append(1)
    else : 
        temporaire.append(0)

data_gen['Syndrome_coronaire_aigu_indicatrice'] = temporaire
data_gen = data_gen.drop(columns = ['Syndrome_coronaire_aigu_indicatrice'])
data_gen.head()

,ind,annee,patho_niv1,patho_niv2,patho_niv3,top,cla_age_5,sexe,region,dept,ntop,npop,prev,niveau_prioritaire,libelle_classe_age,libelle_sexe,tri
0,1,2020.0,Traitements du risque vasculaire (hors patholo...,Traitements hypolipémiants (hors pathologies),Traitements hypolipémiants (hors pathologies),top_FHypoLi_med,80-84,1.0,1,971,290.0,4210.0,6.912,"2,3",de 80 à 84 ans,hommes,51.0
1,2,2020.0,Traitements du risque vasculaire (hors patholo...,Traitements hypolipémiants (hors pathologies),Traitements hypolipémiants (hors pathologies),top_FHypoLi_med,80-84,1.0,3,973,40.0,650.0,5.864,"2,3",de 80 à 84 ans,hommes,51.0
2,3,2020.0,Traitements du risque vasculaire (hors patholo...,Traitements hypolipémiants (hors pathologies),Traitements hypolipémiants (hors pathologies),top_FHypoLi_med,80-84,1.0,4,999,410.0,5630.0,7.340,"2,3",de 80 à 84 ans,hommes,51.0
3,4,2020.0,Traitements du risque vasculaire (hors patholo...,Traitements hypolipémiants (hors pathologies),Traitements hypolipémiants (hors pathologies),top_FHypoLi_med,80-84,1.0,6,999,20.0,350.0,4.913,"2,3",de 80 à 84 ans,hommes,51.0
4,5,2020.0,Traitements du risque vasculaire (hors patholo...,Traitements hypolipémiants (hors pathologies),Traitements hypolipémiants (hors pathologies),top_FHypoLi_med,80-84,1.0,11,75,2960.0,20820.0,14.221,"2,3",de 80 à 84 ans,hommes,51.0


In [130]:
def creation_indicatrice (colonne_de_depart, nom_de_maladie) : 
    temporaire = []
    for i in colonne_de_depart :
        if i == f"{nom_de_maladie}" :
            temporaire.append(1)
        else :
            temporaire.append(0)
    data_gen[f"{nom_de_maladie}_ind"] = temporaire

In [131]:
for i in liste_diff_patho3 : 
    creation_indicatrice(colonne_de_depart = data_gen['patho_niv3'], nom_de_maladie = i)

In [132]:
data_gen.head()

,ind,annee,patho_niv1,patho_niv2,patho_niv3,top,cla_age_5,sexe,region,dept,...,Épilepsie_ind,Autres troubles psychiatriques_ind,Démences (dont maladie d'Alzheimer)_ind,Maladie de Parkinson_ind,Myopathie ou myasthénie_ind,Paraplégie_ind,Sclérose en plaques_ind,Transplantation rénale_ind,Suivi de transplantation rénale_ind,None_ind
0,1,2020.0,Traitements du risque vasculaire (hors patholo...,Traitements hypolipémiants (hors pathologies),Traitements hypolipémiants (hors pathologies),top_FHypoLi_med,80-84,1.0,1,971,...,0,0,0,0,0,0,0,0,0,0
1,2,2020.0,Traitements du risque vasculaire (hors patholo...,Traitements hypolipémiants (hors pathologies),Traitements hypolipémiants (hors pathologies),top_FHypoLi_med,80-84,1.0,3,973,...,0,0,0,0,0,0,0,0,0,0
2,3,2020.0,Traitements du risque vasculaire (hors patholo...,Traitements hypolipémiants (hors pathologies),Traitements hypolipémiants (hors pathologies),top_FHypoLi_med,80-84,1.0,4,999,...,0,0,0,0,0,0,0,0,0,0
3,4,2020.0,Traitements du risque vasculaire (hors patholo...,Traitements hypolipémiants (hors pathologies),Traitements hypolipémiants (hors pathologies),top_FHypoLi_med,80-84,1.0,6,999,...,0,0,0,0,0,0,0,0,0,0
4,5,2020.0,Traitements du risque vasculaire (hors patholo...,Traitements hypolipémiants (hors pathologies),Traitements hypolipémiants (hors pathologies),top_FHypoLi_med,80-84,1.0,11,75,...,0,0,0,0,0,0,0,0,0,0


In [133]:
data_gen.shape

(4057201, 77)

In [104]:
#perte d un grand nombre de lignes 
#pb a regler
#pb regle, venait du test pour essayer de comprendre la variable npop

In [134]:
liste_var = data_gen.columns[17:]
liste_bis = []
for i in liste_var : 
    liste_bis.append(i)


In [135]:
indicatrice_patho_niv3 = liste_bis 
print(indicatrice_patho_niv3)

['Traitements hypolipémiants (hors pathologies)_ind', 'nan_ind', "Traitements antidépresseurs ou régulateurs de l'humeur (hors pathologies)_ind", 'Séjours en hospitalisation complète pour prise en charge de la Covid-19_ind', 'Maladies respiratoires chroniques (hors mucoviscidose)_ind', 'Maternité (avec ou sans pathologies)_ind', 'Troubles psychotiques_ind', 'Traitements antihypertenseurs (hors pathologies)_ind', 'Troubles addictifs_ind', "Troubles névrotiques et de l'humeur_ind", "Troubles psychiatriques ayant débuté dans l'enfance_ind", 'Déficience mentale_ind', 'Autres affections de longue durée (dont 31 et 32)_ind', 'Hospitalisations ponctuelles (avec ou sans pathologies, traitements ou maternité)_ind', 'Traitements anxiolytiques (hors pathologies)_ind', 'Autres cancers actifs_ind', 'Autres cancers sous surveillance_ind', 'Traitements hypnotiques (hors pathologies)_ind', 'Traitements neuroleptiques (hors pathologies)_ind', 'Cancer colorectal actif_ind', 'Cancer du sein de la femme s

In [136]:
liste_auxil = data_gen['cla_age_5'].unique()
print(liste_auxil)

['80-84' '85-89' '90-94' '95et+' 'tsage' '00-04' '05-09' '10-14' '15-19'
 '20-24' '25-29' '30-34' '35-39' '40-44' '45-49' '50-54' '55-59' '60-64'
 '65-69' '70-74' '75-79' nan]


In [137]:
#afin de pouvoir clusteriser par la suite, nous allons creer des classes d age qui sont "discretes"
#nous faisons ce choix car les classes d age sont relativement homogene et en prenant la moyenne des deux 
#bornes pour les classes, nous gardons l idee d une distance d age qui pourrait etre utile par la suite

In [148]:
test_de_split = data_gen['cla_age_5'][1].split('-')
print(test_de_split)
test_de_split[1] = int(test_de_split[1])
type(test_de_split[1])

['80', '84']


int

In [159]:
classe_age = []
for i in data_gen['cla_age_5'] : 
    if i == '95et+' : 
        age = 100
    elif i == 'tsage' : 
        age = np.nan
    elif i == np.nan : 
        age = np.nan
    else : 
        temp = i.split('-')
        temp[0] = int(temp[0])
        temp[1] = int(temp[1])
        age = (temp[1] + temp[0])/2
    classe_age.append(age)

AttributeError: 'float' object has no attribute 'split'

In [ ]:
#pb a regler, il manque un truc pour eviter que ca bug fort
#probablement type a regler ou categorie a revoir